<a href="https://colab.research.google.com/github/saifEEE01/NLP_course/blob/main/Exam_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import collections
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
#Downloading NLTK data files
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
#Faq file preparation

faq_data = {
    "Do you offer delivery?": "Yes, we provide delivery service for all orders above 500 BDT.",
    "What payment methods do you accept?": "We accept cash on delivery, bKash, and bank transfer.",
    "Do you deliver outside of Dhaka?": "Currently, we only deliver within Dhaka city.",
    "What is your operating hour?": "We are open from 9 AM to 9 PM, Saturday to Thursday.",
    "Where are you located?": "We are located at 123 Main Road, Dhaka.",
    "Can I return a product?": "You can return a product within 7 days of delivery.",
    "How can I contact customer support?": "You can contact us via email at support@example.com or call 123-456-7890."
}

In [10]:
#Convert all keys to lowercase for case-insensitive matching
faq_data = {n.lower(): p for n, p in faq_data.items()}
#Tokenization and vectorization
faq_questions = list(faq_data.keys())
faq_answers = list(faq_data.values())
counter = collections.Counter()
maxlen = 0
for question in faq_data.keys():
    tokens = word_tokenize(question)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    counter.update(tokens)
    maxlen = max(maxlen, len(tokens))

In [11]:
#TF-IDF vectorization
tfidf=TfidfVectorizer(ngram_range=(1,3),use_idf=True,tokenizer=lambda x:x.split()) #trigram use
vectorizer = TfidfVectorizer().fit(faq_questions)
faq_vectors = vectorizer.transform(faq_questions)

In [22]:
#Faq answer function
def get_faq_answer(user_query: str) -> str:
    if not user_query.strip():
        return "Please enter a valid question."

    # Vectorize user query
    query_vec = vectorizer.transform([user_query])
    #Compute cosine similarity
    similarities = cosine_similarity(query_vec, faq_vectors).flatten()

    #Find best match and apply threshold
    best_idx = np.argmax(similarities)
    best_score = similarities[best_idx]
    threshold = 0.6  # Adjust threshold based on testing
    if best_score < threshold:
        return "Sorry, I couldn’t find a suitable answer to your question."
    else:
        return faq_data[faq_questions[best_idx]]

In [23]:
#Testing examples
if __name__ == "__main__":
    test_queries = [
        "Do you offer delivery?",
        "Where are you located?",
        "Where are you situated?",
        "Can I return a product?",
        "Do you sell mobile phones?",
        "How to contact customer support?",
    ]

    for query in test_queries:
        print(f"Q: {query}")
        print(f"A: {get_faq_answer(query)}\n")

Q: Do you offer delivery?
A: Yes, we provide delivery service for all orders above 500 BDT.

Q: Where are you located?
A: We are located at 123 Main Road, Dhaka.

Q: Where are you situated?
A: We are located at 123 Main Road, Dhaka.

Q: Can I return a product?
A: You can return a product within 7 days of delivery.

Q: Do you sell mobile phones?
A: Sorry, I couldn’t find a suitable answer to your question.

Q: How to contact customer support?
A: You can contact us via email at support@example.com or call 123-456-7890.



NameError: name 'query_vec' is not defined